In [0]:
!pip install import-ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=9aaed36cd78ea14c902ee3a20329e3741c4e90ce290176ec838e007ecbe9b0bf
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd "/content/drive/My Drive/MRNET for Knee Diagnosis"
import import_ipynb

/content/drive/My Drive/MRNET for Knee Diagnosis


In [0]:
import dataset_loader as loader
import models as models

importing Jupyter notebook from dataset_loader.ipynb
2.2.0
importing Jupyter notebook from models.ipynb
2.2.0


In [0]:
# Import the dependencies
import tensorflow as tf
from tensorflow import keras
import numpy as np
 
import os
import gc
gc.enable()

import matplotlib as mpl
import matplotlib.pyplot as plt
 
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
 
#2.2.0
print(tf.__version__)

2.2.0


In [0]:
root_dir = "/content/drive/My Drive/MRNET for Knee Diagnosis"
dataset_dir = root_dir + "/tmp/MRNet-v1.0"
task = "acl"              # choose the type of the task (acl, meniscus or abnormal)
planes = ["sagittal", "coronal", "axial"]

In [0]:
trainLoader_sagittal = loader.FullDataGenerator(dataset_dir, task,planes[0], train= True)
trainLoader_coronal = loader.FullDataGenerator(dataset_dir, task,planes[1], train= True)
trainLoader_axial = loader.FullDataGenerator(dataset_dir, task,planes[2], train= True)

In [0]:
def extract_predictions(models, dataLoaders):
  size = dataLoaders[0].__len__()

  X = np.empty((size, 3), dtype=np.float32)

  X[:, 0] = models[0].predict(dataLoaders[0], verbose=1, workers=11)[:,0]
  X[:, 1] = models[1].predict(dataLoaders[1], verbose=1, workers=11)[:,0]
  X[:, 2] = models[2].predict(dataLoaders[2], verbose=1, workers=11)[:,0]
  Y = np.array(dataLoaders[0].labels)

  return X, Y

In [0]:
name = "Michael" #<-------------------- enter your name plz.
check_point_path = root_dir + "/checkpoint/"+name+"/full_model-{0}".format(task)

In [0]:
print("loading sagittal model ....")
MRNet_sagittal = keras.models.load_model(root_dir + "/checkpoint/"+name+"/model-{0}-{1}".format(task,"sagittal"))
MRNet_sagittal._name = MRNet_sagittal._name + "_sagittal"

loading sagittal model ....


In [0]:
print("loading coronal model ....")
MRNet_coronal = keras.models.load_model(root_dir + "/checkpoint/"+name+"/model-{0}-{1}".format(task,"coronal"))
MRNet_coronal._name = MRNet_coronal._name + "_coronal"

loading coronal model ....


In [0]:
print("loading axial model ....")
MRNet_axial = keras.models.load_model(root_dir + "/checkpoint/"+name+"/model-{0}-{1}".format(task,"axial"))
MRNet_axial._name = MRNet_axial._name + "_axial"

loading axial model ....


In [0]:
X, Y = extract_predictions([MRNet_sagittal, MRNet_coronal, MRNet_axial], [trainLoader_sagittal, trainLoader_coronal, trainLoader_axial])

1130/1130 [==============================] - 732s 648ms/step


In [0]:
validLoader_sagittal = loader.FullDataGenerator(dataset_dir, task,planes[0], train= False)
validLoader_coronal = loader.FullDataGenerator(dataset_dir, task,planes[1], train= False)
validLoader_axial = loader.FullDataGenerator(dataset_dir, task,planes[2], train= False)

In [0]:
X_valid, Y_valid = extract_predictions([MRNet_sagittal, MRNet_coronal, MRNet_axial], [validLoader_sagittal, validLoader_coronal, validLoader_axial])

120/120 [==============================] - 77s 638ms/step


In [0]:
logisticRegression = models.LogisticRegression().getModel()

In [0]:
metrics = [                                           
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.AUC(name='auc')]
callbacks = [                                         
      keras.callbacks.EarlyStopping(                         
        monitor='val_loss', 
        verbose=1,
        patience= 5,
        restore_best_weights=True
        )]
logisticRegression.compile(optimizer= keras.optimizers.Adam(), loss= keras.losses.BinaryCrossentropy(), metrics= metrics)

In [0]:
train_history= logisticRegression.fit(X, Y , epochs=20, callbacks= callbacks, validation_data=(X_valid, Y_valid), batch_size= 1)

Epoch 1/20
1130/1130 [==============================] - 6s 5ms/step - loss: 0.3199 - accuracy: 0.9708 - auc: 0.9825 - val_loss: 0.5948 - val_accuracy: 0.6500 - val_auc: 0.8486
Epoch 2/20
1130/1130 [==============================] - 6s 5ms/step - loss: 0.2449 - accuracy: 0.9708 - auc: 0.9885 - val_loss: 0.5996 - val_accuracy: 0.6583 - val_auc: 0.8621
Epoch 3/20
1130/1130 [==============================] - 6s 5ms/step - loss: 0.2029 - accuracy: 0.9726 - auc: 0.9870 - val_loss: 0.5962 - val_accuracy: 0.6667 - val_auc: 0.8699
Epoch 4/20
1130/1130 [==============================] - 6s 5ms/step - loss: 0.1739 - accuracy: 0.9752 - auc: 0.9897 - val_loss: 0.5940 - val_accuracy: 0.6917 - val_auc: 0.8810
Epoch 5/20
1130/1130 [==============================] - 6s 5ms/step - loss: 0.1531 - accuracy: 0.9770 - auc: 0.9904 - val_loss: 0.5968 - val_accuracy: 0.7083 - val_auc: 0.8836
Epoch 6/20
1130/1130 [==============================] - 6s 5ms/step - loss: 0.1378 - accuracy: 0.9770 - auc: 0.9897 - va

In [0]:
evaluate_results = logisticRegression.evaluate(X_valid, Y_valid, batch_size= 1)
for name, value in zip(logisticRegression.metrics_names, evaluate_results):
        print(name, ': ', value)

120/120 [==============================] - 0s 4ms/step - loss: 0.5940 - accuracy: 0.6917 - auc: 0.8810
loss :  0.5939998626708984
accuracy :  0.6916666626930237
auc :  0.8810325860977173
